In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os.path

# Tensorflow imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

## Model parameters

In [2]:
model_name = 'faceDetectModel_tl'
savedModelPath = '../log/saved_models/faceDetectModel_tl'
tb_log_dir = "../log/tensorboard/faceDetectModel_tl"
cp_filepath = '../log/cps/faceDetectModel_tl/latest_weights.h5'

batch_size = 32
epochs = 10

doDataAugmentation = False
useDropout = False
dropoutRate = 0.25

## Model callbacks

In [3]:
callbacks = [
    # Checkpoint callback    
    keras.callbacks.ModelCheckpoint(      
                    filepath=cp_filepath, 
                    verbose=1, 
                    save_weights_only=False
    ),

    # Tensorboard callback
    tf.keras.callbacks.TensorBoard(log_dir=tb_log_dir, histogram_freq=1),

    # Early Stopping callback
    keras.callbacks.EarlyStopping(
                    monitor="val_loss",
                    patience=2,
                    verbose=1,
                )
]

## Data parameters

In [4]:
image_height = 224
image_width = 224

## Data Augmentation

In [5]:
data_augmentation = keras.Sequential(
    [
        keras.layers.RandomFlip("horizontal"),
        keras.layers.RandomRotation(0.1),
    ]
)

## Creating the training dataset

In [6]:
# https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory

face_training_ds = keras.utils.image_dataset_from_directory(
    "../data/training/",
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(image_height, image_width),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='training',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=True,
)

face_training_ds.cache().batch(batch_size).prefetch(buffer_size=10)

Found 28312 files belonging to 2 classes.
Using 22650 files for training.


<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float32, name=None))>

## Creating the validation dataset

In [7]:
validation_ds = keras.utils.image_dataset_from_directory(
    "../data/training/",
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(image_height, image_width),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='validation',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=True,
)

Found 28312 files belonging to 2 classes.
Using 5662 files for validation.


## Creating the test dataset

In [8]:
test_ds = keras.utils.image_dataset_from_directory(
    "../data/test/",
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(image_height, image_width),
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=True,
)

test_ds.cache().batch(batch_size).prefetch(buffer_size=10)

Found 353 files belonging to 2 classes.


<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float32, name=None))>

## Load the base model

In [9]:
# Loading either the ResNet50 architecture model or the previously saved model, and freeze it for transfer learning
base_model = ResNet50(
                include_top=False,
                weights='imagenet',  # Load weights pre-trained on ImageNet.
                input_shape=(image_height, image_width, 3)
                )
base_model.trainable = False

## Creating a new model

In [10]:
inputs = keras.Input(shape=(image_height, image_width, 3))

# Data Augmentation on input
if(doDataAugmentation):
    inputs = data_augmentation(inputs)

# Running base model in inference mode
x = base_model(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)

# Add Dropout layer
if(useDropout):
    x = keras.layers.Dropout(dropoutRate)(x)

# Final layer for binary classification
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs, outputs)

# Loading latest saved weights into model
if(os.path.isfile(cp_filepath)):
    print('Loading weights...')
    model.load_weights(filepath=cp_filepath)

## Training the new model with new data

In [11]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
history = model.fit(
            face_training_ds, 
            epochs=epochs, 
            callbacks=callbacks, 
            validation_data=validation_ds
        )

Epoch 1/10
708/708 [==============================] - ETA: 0s - loss: 0.0314 - accuracy: 0.9896
Epoch 1: saving model to ../log/cps/faceDetectModel_tl\latest_weights.h5
708/708 [==============================] - 1026s 1s/step - loss: 0.0314 - accuracy: 0.9896 - val_loss: 0.0179 - val_accuracy: 0.9936
Epoch 2/10
708/708 [==============================] - ETA: 0s - loss: 0.0150 - accuracy: 0.9953
Epoch 2: saving model to ../log/cps/faceDetectModel_tl\latest_weights.h5
708/708 [==============================] - 1011s 1s/step - loss: 0.0150 - accuracy: 0.9953 - val_loss: 0.0157 - val_accuracy: 0.9951
Epoch 3/10
708/708 [==============================] - ETA: 0s - loss: 0.0116 - accuracy: 0.9962
Epoch 3: saving model to ../log/cps/faceDetectModel_tl\latest_weights.h5
708/708 [==============================] - 978s 1s/step - loss: 0.0116 - accuracy: 0.9962 - val_loss: 0.0187 - val_accuracy: 0.9942
Epoch 4/10
708/708 [==============================] - ETA: 0s - loss: 0.0083 - accuracy: 0.9975

## Save the model

In [12]:
model.save(savedModelPath)

INFO:tensorflow:Assets written to: ../log/saved_models/faceDetectModel_tl\assets


INFO:tensorflow:Assets written to: ../log/saved_models/faceDetectModel_tl\assets


## Test model with test dataset

In [13]:
results = model.evaluate(test_ds)

print(f'Loss: {results[0]}; Accuracy: {results[1]}')

12/12 [==============================] - 13s 1s/step - loss: 0.1054 - accuracy: 0.9660
Loss: 0.10540056973695755; Accuracy: 0.9660056829452515


## Predict new images

In [14]:
# https://www.tensorflow.org/tutorials/images/classification

img = tf.keras.utils.load_img(
    '../data/test/NoFace/000001.jpg', target_size=(image_height, image_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

preds = model.predict(img_array)
score = preds[0][0]

if(score > 0.5):
    print("This image contains no face. ({:.2f} percent confidence)".format(100 * score))
else:
    print("This image contains a face. ({:.2f} percent confidence)".format(100 * (1-score)))

1/1 [==============================] - 1s 1s/step
This image contains no face. (100.00 percent confidence)
